In [ ]:
%matplotlib inline
#%matplotlib notebook
from datetime import datetime
import numpy as np
import os
import matplotlib
#matplotlib.rcParams['savefig.dpi'] = 200
matplotlib.rcParams['figure.figsize'] = (12.0, 8.0)
matplotlib.rcParams["font.size"] = 20
import matplotlib.pyplot as plt

In [ ]:
class Logfile():
    def __init__(self, dirname, results):
        self.dirname = dirname
        dtimes, steps, ts, dts = zip(*results)
        dtimes = list(dtimes)
        dt_steps = [(dtimes[i+1]-dtimes[i]).total_seconds() for i in range(len(dtimes)-1)]
        
        self.dtimes = dtimes
        self.steps = steps
        self.ts = ts
        self.dts = dts
        self.dt_steps = dt_steps
    

def parse_logfile(fname):
    dirname = fname.split('/')[-2]
    startmarks = []
    results = []
    with open(fname, 'r') as f:
        start = False
        for line in f.readlines():
            if 'FLASH log file:' in line:
                start = True
            if 'Number of MPI tasks:' in line:
                nProc = int(line.split()[-1])
            if 'step: n=' in line:
                dummy, date, time, dummy, dummy, step, t, dt = line.split()
                dtime = datetime.strptime(date+' '+time, '%m-%d-%Y %H:%M:%S.%f')
                step = int(step.lstrip('n='))
                t = float(t.lstrip('t='))
                dt = float(dt.lstrip('dt='))
                results.append([dtime, step, t, dt])
                if start:
                    startmarks.append([step])
                    start = False
                    
    return Logfile(dirname, results)


In [ ]:
    dtimes, steps, ts, dts = zip(*results)
    dtimes = list(dtimes)
    dt_steps = [(dtimes[i+1]-dtimes[i]).total_seconds() for i in range(len(dtimes)-1)]
    print(np.average(dt_steps))
    plt.clf()
    plt.scatter(steps[1:], dt_steps, s=5, linewidth=0, label='t/step')

    
    # Plot start steps
    plt.vlines(startmarks,0.2,1000, color='grey', linestyles='--', alpha=0.5,)

    
    plt.title()
    plt.show()

In [ ]:
fnames = [#'/d/d9/ychen/stampede2_test/stampede2_16_1024/MHD_Jet.log',
          #'/d/d9/ychen/stampede2_test/stampede2_16_1024_b/MHD_Jet.log',
          '/d/d9/ychen/stampede2_test/stampede2_32_2048/MHD_Jet.log',
          '/d/d9/ychen/stampede2_test/stampede2_32_1536/MHD_Jet.log']
          #'/d/d9/ychen/stampede2_test/stampede_32_512/MHD_Jet.log',
          #'/d/d9/ychen/stampede2_test/aci_8_160/MHD_Jet.log']

for fname in fnames:
    log = parse_logfile(fname)
    n_node = int(fname.strip('/MHD_Jet.log').split('_')[-2])
    n_cpu = int(fname.strip('/MHD_Jet.log').split('_')[-2])
    #plt.scatter(log.steps[1:], log.dt_steps, s=5, linewidth=1, label=log.dirname)
    plt.plot(log.steps[1:], 1.0/np.array(log.dt_steps)*60*60/n_node, label=log.dirname)
    print('%18s: %i nodes, %.4f steps/min %.4f steps/hour/node' % 
          (log.dirname, n_node, np.average(1.0/np.array(log.dt_steps[:100])*60), 
           np.average(1.0/np.array(log.dt_steps[:100])*60*60)/n_node))

plt.ylim(0, 15)
plt.xlabel('# of step')
#plt.semilogy()
plt.ylabel('speed (steps/hour/node)')
plt.legend(loc=3)


In [ ]:
#%pdb
fnames = ['/d/d9/ychen/stampede2_test/stampede2_16_1024/MHD_Jet.log',
          #'/d/d9/ychen/stampede2_test/stampede2_16_1024_b/MHD_Jet.log',
          '/d/d9/ychen/stampede2_test/stampede2_32_2048/MHD_Jet.log']
          #'/d/d9/ychen/stampede2_test/stampede2_32_1536/MHD_Jet.log',
          #'/d/d9/ychen/stampede2_test/stampede_32_512/MHD_Jet.log',
          #'/d/d9/ychen/stampede2_test/aci_8_160/MHD_Jet.log']

for fname in fnames:
    log = parse_logfile(fname)
    n_node = int(fname.strip('/MHD_Jet.log').split('_')[-2])
    n_cpu = int(fname.strip('/MHD_Jet.log').split('_')[-2])
    #plt.scatter(log.steps[1:], log.dt_steps, s=5, linewidth=1, label=log.dirname)
    plt.plot(log.steps[1:], 1.0/np.array(log.dt_steps)*60*60/n_node, label=log.dirname)
    print('%18s: %i nodes, %.4f steps/min %.4f steps/hour/node' % 
          (log.dirname, n_node, np.average(1.0/np.array(log.dt_steps[:100])*60), 
           np.average(1.0/np.array(log.dt_steps[:100])*60*60)/n_node))

plt.ylim(0, 15)
plt.xlabel('# of step')
#plt.semilogy()
plt.ylabel('speed (steps/hour/node)')
plt.legend(loc=3)


In [ ]:
fname.strip('/MHD_Jet.log').split('_')[-2]

In [ ]:
#%pdb
fnames = ['/d/d9/ychen/stampede2_test/stampede2_16_1024/MHD_Jet.log',
          '/d/d9/ychen/stampede2_test/stampede2_16_1024_b/MHD_Jet.log']

for fname in fnames:
    log = parse_logfile(fname)
    #plt.scatter(log.steps[1:], log.dt_steps, s=5, linewidth=1, label=log.dirname)
    plt.plot(log.steps[1:], 1.0/np.array(log.dt_steps)*60, label=log.dirname)
    print('%18s: %.4f steps/min' % (log.dirname, np.average(1.0/np.array(log.dt_steps[:100])*60)))

plt.ylim(0, 7)
plt.xlabel('# of step')
#plt.semilogy()
plt.ylabel('speed (steps/min)')
plt.legend(loc=3)


In [ ]:
def get_nozzledata(dirname):
    try:
        nozzledata = np.genfromtxt(os.path.join(dirname, 'nozzleVec.dat'))
        
        ttnoz = nozzledata[:,1]
        xx = nozzledata[:,2]
        yy = nozzledata[:,3]
        zz = nozzledata[:,4]
        thetas = np.arccos(zz)/np.pi*180
        phis = np.arctan2(yy, xx)
        return ttnoz, xx, yy, zz, thetas, phis
    except:
        pass

def get_dat(dirname):
    try:
        data = np.genfromtxt(os.path.join(dirname, 'MHD_Jet.dat'))
    except:
        data = np.genfromtxt(os.path.join(dirname, 'MHD_Jet_10Myr.dat'))
    tt = data[:,0]
    Masses = data[:,1]
    Etots = data[:,5]
    Ekins = data[:,6]
    Eints = data[:,7] 
    Emags = data[:,8]
    Ptots = (Etots[1:]-Etots[:-1])/(tt[1:]-tt[:-1])
    Pkins = (Ekins[1:]-Ekins[:-1])/(tt[1:]-tt[:-1])
    Pints = (Eints[1:]-Eints[:-1])/(tt[1:]-tt[:-1])
    #print max(Ptots), max(Pkins), max(Pints)
    return tt, Masses, Etots, Ekins, Eints, Emags, Ptots, Pkins, Pints

In [ ]:
#dirname = '/home/ychen/data/0529_L45_M10_b1_h1'
#dirname = '/home/ychen/d9/FLASH4/stampede/0916_L45_M10_b1_hinf_10Myr/'
#dirnames = ['/home/ychen/data/0529_L45_M10_b1_h1',\
#            '/home/ychen/data/0605_L45_M10_b1_h0',\
#            '/home/ychen/data/0605_L45_M10_b1_hinf_all',
#            '/home/ychen/data/0602_L45_M10_hydro_all']
dirnames = ['/home/ychen/data/0only_0529_h1',\
            '/home/ychen/data/0only_1106_M3_h1',\
            '/home/ychen/data/0only_1110_h0_rerun',\
            '/home/ychen/data/0only_1111_M10_b01',\
            '/home/ychen/data/0only_1204_M24_b01'
            ]
table = []
for dirname in dirnames:
    table.append(get_dat(dirname))

In [ ]:
names = [dirname.split('/')[-1] for dirname in dirnames]
colors = ['yellow', 'r', 'b', 'g', 'cyan']
plt.figure(figsize=(10,8))
for i in range(len(table)):
    tt, Masses, Etots, Ekins, Eints, Emags, Ptots, Pkins, Pints = table[i]
    plt.plot(tt[1:]/3.154E13, (tt[1:]-tt[:-1]), c=colors[i], label=names[i], lw=0, marker='.', markersize=1, alpha=0.7)


#plt.scatter(tt[:], Etots[:], s=1, linewidth=0, label='dt')
#plt.ylim(2.563E62, 2.563E62+9E59)
#plt.ylim(0, 4E58)
plt.ylim(0,9E9)
plt.xlim(0, 25)
#plt.semilogy()
plt.ylabel('dt')
plt.xlabel('t (Myr)')
plt.legend(loc=2)
plt.show()